<a href="https://colab.research.google.com/github/fatemafaria142/Evaluating-LLMs-Generated-Food-Recipes-in-Comparison-to-Established-Reference-Recipes/blob/main/Recipe_Generation_using_Mistral_7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Install Required Packages**

In [1]:
!pip install accelerate peft bitsandbytes transformers trl datasets torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.2 MB/s eta 0:00:00


In [2]:
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


### **Dataset Link:** https://huggingface.co/datasets/recipe_nlg

In [3]:
import pandas as pd

# Load the original dataset
dataset = pd.read_csv("/content/drive/MyDrive/Recipe Generation /full_dataset.csv")

# Extract the first 3000 rows
subset_dataset = dataset.head(3000)

# Save the subset to a new CSV file
subset_dataset.to_csv("/content/drive/MyDrive/Recipe Generation /recipe_generation.csv", index=False)


In [7]:
import pandas as pd
instruct_tune_dataset = pd.read_csv("/content/drive/MyDrive/Recipe Generation /recipe_generation.csv")
instruct_tune_dataset.head()

,Unnamed: 0,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


In [8]:
length_of_data = len(instruct_tune_dataset)
# Print the length
print(length_of_data)

3000


In [11]:
from datasets import Dataset, DatasetDict

# Create a DatasetDict for instruct_tune_dataset
instruct_tune_dataset = DatasetDict()

# Select 1500 rows for the 'train' split and only keep 'ingredients' and 'directions'
instruct_tune_dataset["train"] = Dataset.from_pandas(dataset[["ingredients", "directions"]].head(2500))

# Select 200 rows for the 'test' split and only keep 'ingredients' and 'directions'
instruct_tune_dataset["test"] = Dataset.from_pandas(dataset[["ingredients", "directions"]].tail(200))

# Print the updated instruct_tune_dataset
print(instruct_tune_dataset)

DatasetDict({
    train: Dataset({
        features: ['ingredients', 'directions'],
        num_rows: 2500
    })
    test: Dataset({
        features: ['ingredients', 'directions'],
        num_rows: 200
    })
})


### **We will use just a small subset of the data for this training example**

In [12]:
instruct_tune_dataset

DatasetDict({
    train: Dataset({
        features: ['ingredients', 'directions'],
        num_rows: 2500
    })
    test: Dataset({
        features: ['ingredients', 'directions'],
        num_rows: 200
    })
})

In [13]:
def create_prompt(sample):
    """
    Update the prompt template:
    Combine both the prompt and input into a single column.
    """
    bos_token = ""
    eos_token = ""

    # Use a predefined template for instructions
    instructions_template = "Imagine you're a chef, and you've got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish."



    full_prompt = ""
    full_prompt += bos_token
    full_prompt += "### Instructions:"
    full_prompt += "\n" + instructions_template
    full_prompt += "\n\n### Input:"
    full_prompt += "\n" + sample['ingredients']
    full_prompt += "\n\n### Chef:"
    full_prompt += "\n" + sample['directions']
    full_prompt += eos_token

    return full_prompt

In [14]:
create_prompt(instruct_tune_dataset["train"][0])

'### Instructions:\nImagine you\'re a chef, and you\'ve got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish.\n\n### Input:\n["1 c. firmly packed brown sugar", "1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 c. broken nuts (pecans)", "2 Tbsp. butter or margarine", "3 1/2 c. bite size shredded rice biscuits"]\n\n### Chef:\n["In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine.", "Stir over medium heat until mixture bubbles all over top.", "Boil and stir 5 minutes more. Take off heat.", "Stir in vanilla and cereal; mix well.", "Using 2 teaspoons, drop and shape into 30 clusters on wax paper.", "Let stand until firm, about 30 minutes."]'

In [15]:
create_prompt(instruct_tune_dataset["train"][1])

'### Instructions:\nImagine you\'re a chef, and you\'ve got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish.\n\n### Input:\n["1 small jar chipped beef, cut up", "4 boned chicken breasts", "1 can cream of mushroom soup", "1 carton sour cream"]\n\n### Chef:\n["Place chipped beef on bottom of baking dish.", "Place chicken on top of beef.", "Mix soup and cream together; pour over chicken. Bake, uncovered, at 275\\u00b0 for 3 hours."]'

In [16]:
create_prompt(instruct_tune_dataset["train"][2])

'### Instructions:\nImagine you\'re a chef, and you\'ve got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish.\n\n### Input:\n["2 (16 oz.) pkg. frozen corn", "1 (8 oz.) pkg. cream cheese, cubed", "1/3 c. butter, cubed", "1/2 tsp. garlic powder", "1/2 tsp. salt", "1/4 tsp. pepper"]\n\n### Chef:\n["In a slow cooker, combine all ingredients. Cover and cook on low for 4 hours or until heated through and cheese is melted. Stir well before serving. Yields 6 servings."]'

### **Initializing the Model**
* Load the model using a 4-bit configuration, employing double quantization, and set bfloat16 as the compute data type.

* Notably, we opt for the instruct-tuned model in this instance rather than the base model. It's worth mentioning that fine-tuning a base model necessitates a more substantial amount of data!

In [17]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
    )


* https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1

In [18]:
mode_id = "mistralai/Mistral-7B-Instruct-v0.1"

In [19]:
model = AutoModelForCausalLM.from_pretrained(
        mode_id, quantization_config=bnb_config, device_map="auto", use_cache=False
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [20]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

### **Let's example how well the model does at this task currently:**

In [21]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=1024, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0].replace(prompt, "")

In [23]:
prompt = """### Instruction:
Imagine you're a chef, and you've got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish.### Input:
'''["1 small jar chipped beef, cut up", "4 boned chicken breasts", "1 can cream of mushroom soup", "1 carton sour cream"]'''

### Chef:"""

In [24]:
generate_response(prompt, model)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


"<s> \nHello there! Today, we'll be making a delicious Creamed Mushroom Chicken dish. This dish will be a comforting and satisfying meal, perfect for a cold and rainy day. First, let's get started with the ingredients. You'll need:\n1. 1 small jar chipped beef, cut up\n2. 4 boned chicken breasts\n3. 1 can cream of mushroom soup\n4. 1 carton sour cream\n\nNow that we've gathered all the ingredients, let's start cooking.\n\nStep 1: Preheat your oven to 350°F.\n\nStep 2: In a large bowl, mix together the chopped beef, chicken breasts, and cream of mushroom soup.\n\nStep 3: Pour the mixture into a greased 9x13 inch baking dish.\n\nStep 4: In a small bowl, mix together the sour cream.\n\nStep 5: Pour the sour cream over the chicken mixture.\n\nStep 6: Make sure all the chicken is fully coated with the sour cream.\n\nStep 7: Place the baking dish in the preheated oven.\n\nStep 8: Bake for 25-30 minutes, or until the chicken is cooked through.\n\nStep 9: Remove the dish from the oven and let 

### **Setting up the Training**
we will be using the `huggingface` and the `peft` library!

In [25]:
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")


* we need to prepare the model to be trained in 4bit so we will use the  **`prepare_model_for_kbit_training`** function from peft




In [26]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

# **Training Hyperparameters**
The choice of hyperparameters is contingent upon the desired training duration. Pay special attention to the following key factors:

* `num_train_epochs/max_steps:` Dictates the number of iterations over the data. Exercise caution, as an excessive number may lead to overfitting!

* `learning_rate:` Governs the convergence speed of the model. Adjust this parameter judiciously for optimal results.

In [27]:
from transformers import TrainingArguments
output_model= "mistral_instruct_generation"
training_arguments = TrainingArguments(
        output_dir=output_model,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=1,
        max_steps=50,
        fp16=True,
)


### **Setting up the trainer**

`max_seq_length`: Context window size


In [28]:
from trl import SFTTrainer

max_seq_length = 1024

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=True,
  formatting_func=create_prompt, # this will aplly the create_prompt mapping to all training and test dataset
  args=training_arguments,
  train_dataset=instruct_tune_dataset["train"],
  eval_dataset=instruct_tune_dataset["test"]
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:304: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


### **Training starts here**

In [29]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,1.154300
20,0.961300
30,0.889700
40,0.883100
50,0.908600


TrainOutput(global_step=50, training_loss=0.9593903350830079, metrics={'train_runtime': 763.8367, 'train_samples_per_second': 0.262, 'train_steps_per_second': 0.065, 'total_flos': 8741766719078400.0, 'train_loss': 0.9593903350830079, 'epoch': 0.39})

### **Save the model**

In [30]:
trainer.save_model("mistral_instruct_generation")

In [31]:
merged_model = model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:229: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [32]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0]

### **Example No:1**

In [33]:
# Example usage
prompt = '''### Instruction:\n"Imagine you're a chef, and you've got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish." '''
prompt += '''\n### Input:\n["1 small jar chipped beef, cut up", "4 boned chicken breasts", "1 can cream of mushroom soup", "1 carton sour cream"]'''
prompt += "\n\n### Chef:"
response = generate_response(prompt, model)

# Print the response with formatted output
print(response)

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


<s> ### Instruction:
"Imagine you're a chef, and you've got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish." 
### Input:
["1 small jar chipped beef, cut up", "4 boned chicken breasts", "1 can cream of mushroom soup", "1 carton sour cream"]

### Chef:
Here's what you do:

1. First, preheat your oven to 350 degrees Fahrenheit.

2. Take the 1 small jar of chipped beef and cut it up into small pieces. Set it aside.

3. Next, take the 4 boned chicken breasts and place them in a baking dish.

4. Open the can of cream of mushroom soup and pour half of it over the chicken breasts, making sure they are fully coated.

5. Now, take the carton of sour cream and pour the remaining half of the cream of mushroom soup into it. Mix it well.

6. Pour the mixture over the chicken breasts, making sure they are fully covered.

7. Add the chopped chipped beef on top of the sour cream mixture.

8. Place the baking dish in the p

### **Example No:2**

In [34]:
# Example usage
prompt = '''### Instruction:\n"Imagine you're a chef, and you've got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish." '''
prompt += '''\n### Input:\n["2 (16 oz.) pkg. frozen corn", "1 (8 oz.) pkg. cream cheese, cubed", "1/3 c. butter, cubed", "1/2 tsp. garlic powder", "1/2 tsp. salt", "1/4 tsp. pepper"]'''
prompt += "\n\n### Chef:"
response = generate_response(prompt, model)

# Print the response with formatted output
print(response)

<s> ### Instruction:
"Imagine you're a chef, and you've got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish." 
### Input:
["2 (16 oz.) pkg. frozen corn", "1 (8 oz.) pkg. cream cheese, cubed", "1/3 c. butter, cubed", "1/2 tsp. garlic powder", "1/2 tsp. salt", "1/4 tsp. pepper"]

### Chef:
"Great! Today, we'll be making a delicious Corn and Cream Cheese Casserole! Here are the steps:

1. Preheat oven to 350 degrees Fahrenheit.

2. Drain the corn and place it into a large mixing bowl. Add the cream cheese, butter, garlic powder, salt, and pepper. Mix all ingredients well.

3. Pour the mixture into a baking dish and bake for 30-35 minutes, or until the casserole is golden brown on top.

4. Let cool for few minutes before serving, and enjoy your delicious and warming Corn and Cream Cheese Casserole!"</s>


### **Example No:3**

In [36]:
# Example usage
prompt = '''### Instruction:\n"Imagine you're a chef, and you've got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish." '''
prompt += '''\n### Input:\n["1 (17 oz.) can dark sweet pitted cherries", "1/2 c. ginger ale", "1 (6 oz.) pkg. Jell-O cherry flavor gelatin", "2 c. boiling water", "1/8 tsp. almond extract", "1 c. miniature marshmallows"]'''
prompt += "\n\n### Chef:"
response = generate_response(prompt, model)

# Print the response with formatted output
print(response)

<s> ### Instruction:
"Imagine you're a chef, and you've got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish." 
### Input:
["1 (17 oz.) can dark sweet pitted cherries", "1/2 c. ginger ale", "1 (6 oz.) pkg. Jell-O cherry flavor gelatin", "2 c. boiling water", "1/8 tsp. almond extract", "1 c. miniature marshmallows"]

### Chef:
[Here's a recipe you can make with those ingredients!]

[1. Drain the cherries in a colander, reserving the cherry juice in a bowl.
2. In a large bowl, mix together the ginger ale and cherry juice to form the gelatin mixture.
3. Add the cherry flavor gelatin to the gelatin mixture and mix well.
4. Heat the boiling water in a saucepan over medium heat until it begins to simmer.
5. Add the gelatin mixture to the simmering water and stir until the gelatin has completely dissolved.
6. Off the heat, stir in almond extract.
7. Remove from heat and mix in the miniature marshmallows until they